In [1]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

# import Mongo so our webscraper dumps its scraped data without losing it
from pymongo import MongoClient
import pymongo

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [2]:
client = MongoClient('localhost', 27017)
db = client['craigslist_motorcycles']
post_html = db['motorcycle_posts']

In [3]:
pprint.pprint(post_html.find()[5]['html'])

('<!DOCTYPE html>\n'
 '<html class="no-js">\n'
 '<head>\n'
 '    \n'
 '\t<meta charset="UTF-8">\n'
 '\t<meta http-equiv="X-UA-Compatible" content="IE=Edge">\n'
 '\t<meta name="viewport" content="width=device-width,initial-scale=1">\n'
 '\t<meta property="og:site_name" content="craigslist">\n'
 '\t<meta name="twitter:card" content="preview">\n'
 '\t<meta property="og:title" content="2016 Harley Davidson Roadster Sportster '
 'for Sale -...">\n'
 '\t<meta name="description" content="2016 Harley-Davidson Sportster Roadster '
 '(EVO 1202cc) in excellent condition for sale $6,999 OBO....">\n'
 '\t<meta property="og:description" content="2016 Harley-Davidson Sportster '
 'Roadster (EVO 1202cc) in excellent condition for sale $6,999 OBO....">\n'
 '\t<meta property="og:image" '
 'content="https://images.craigslist.org/00707_gaqoQHqMFur_600x450.jpg">\n'
 '\t<meta property="og:url" '
 'content="https://auburn.craigslist.org/mcy/d/auburn-2016-harley-davidson-roadster/7114312352.html">\n'
 '\t<met

In [21]:
db['motorcycle_posts'][0:1000]

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'craigslist_motorcycles'), 'motorcycle_posts.slice(0, 1000, None)')

In [24]:
def motorcycle_post_parser():
    client = MongoClient('localhost', 27017)
    db = client['craigslist_motorcycles']
    post_html = db['motorcycle_posts'][0:100]
    
    title = []
    price = []
    neighborhood = []
    attributes = []
    description = []
    url = []
    model = []
    vin = []
    condition = []
    cryptocurrency = []
    delivery = []
    engine_size = []
    fuel_type = []
    mileage = []
    paint_color = []
    street_legal = []
    title_status = []
    transmission = []
    bike_type = []

    vin_ = 'VIN:'
    condition_ = 'condition:'
    cryptocurrency_ = 'cryptocurrency'
    delivery_ = 'delivery'
    engine_size_ = 'engine displacement'
    fuel_type_ = 'fuel:'
    mileage_ = 'odometer:'
    paint_color_ = 'paint color:'
    street_legal_ = 'street legal'
    title_status_ = 'title status:'
    transmission_ = 'transmission:'
    bike_type_ = 'type:'
#     for _ in post_html.find(): 

#         #get the first page of the Austin motorcycle prices
#         response = _['html']
#         #parse through it and make it readable
#         html_soup = BeautifulSoup(response.text, 'html.parser')
#         #find the total number of posts to find the limit for each page
#         results_num = html_soup.find('div', class_= 'search-legend')
#         #pulled the total count of posts as the upper bound of the pages array
#         results_total = int(results_num.find('span', class_='totalcount').text) 
#         #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
#         pages = np.arange(0, results_total+1, 120)
#         print("{} Posts = {}".format(city, results_total))
#         print("{} Pages = {}".format(city.title(), len(pages)))

    iterations = 0

    for _ in post_html.find():         
        
        title_ = []
        price_ = []
        neighborhood_ = []
        attributes_ = []
        description_ = []
        url_ = []
        #get request      
        
        response = _['html']
        soup = BeautifulSoup(response, 'html.parser')

        try:
            post_title = soup.find('span', id = 'titletextonly')
            title_.append(post_title.text)
        except:
            title_.append(str(post_title))
        else:
            title_.append('')
        
        try:
            post_price = soup.find('span', class_ = 'price').text
            post_price.lstrip('$')
            price_.append(post_price)
        except:
            price_.append('')
        try:
            post_neighborhood = soup.find('small')
            neighborhood_.append(post_neighborhood.text)
        except:
            neighborhood_.append('')
        
        post_attributes = str(soup.find_all('p', attrs = {'class': 'attrgroup'}))
        post_attributes = post_attributes.replace(post_attributes, str(post_attributes.replace('''[<p class="attrgroup">\n<span><b>''', '').replace('''</b></span>\n<br/>\n</p>, ''','').replace('''<p class="attrgroup">\n<span>''', ',').replace(''' <b>''', '').replace('''</b></span>\n<br/>\n<span>''', ',').replace('''</span>\n<br/>\n<span>''', ',').replace('''</b></span>\n<br/>\n</p>]''', '').replace('''<p class="attrgroup">\n<span class="otherpostings">\n<a href="//auburn.craigslist.org/search/sss?userid=193613506">\nmore ads  by this user        </a>\n</span>\n</p>]''', '')))
        
        for attr in post_attributes:
            
            bike_attributes = (post_attributes.split(','))

            vin_res = [i for i in bike_attributes if vin_ in i]
            condition_res = [i for i in bike_attributes if condition_ in i]
            cryptocurrency_res = [i for i in bike_attributes if cryptocurrency_ in i]
            delivery_res = [i for i in bike_attributes if delivery_ in i]
            engine_size_res = [i for i in bike_attributes if engine_size_ in i]
            fuel_type_res = [i for i in bike_attributes if fuel_type_ in i]
            mileage_res = [i for i in bike_attributes if mileage_ in i]
            paint_color_res = [i for i in bike_attributes if paint_color_ in i]
            street_legal_res = [i for i in bike_attributes if street_legal_ in i]
            title_status_res = [i for i in bike_attributes if title_status_ in i]
            transmission_res = [i for i in bike_attributes if transmission_ in i]
        #     transmission_res = transmission_res.rstrip('<p class="attrgroup">\n<span class="otherpostings">\n<a href="//auburn.craigslist.org/search/sss?userid=193613506">\nmore ads  by this user        </a>\n</span>\n</p>]')
            bike_type_res = str(i for i in bike_attributes if bike_type_ in i)

            model.append(bike_attributes[0])
            vin.append(vin_res)
            condition.append(condition_res)
            cryptocurrency.append((cryptocurrency_res))
            delivery.append(delivery_res)
            engine_size.append(engine_size_res)
            fuel_type.append(fuel_type_res)
            mileage.append(mileage_res)
            paint_color.append(paint_color_res)
            street_legal.append(street_legal_res)
            title_status.append(title_status_res)
            transmission.append(transmission_res)
            bike_type.append(bike_type_res)
    
        try:
            post_description = soup.find('section', id = 'postingbody')
            description_.append(post_description.text)
        except:
            description_.append(post_description)
        else:
            description_.append('')
        
        url_.append(_['_id'])
        
        
        iterations += 1
        
        title.append(title_)
        price.append(price_)
        neighborhood.append(neighborhood_)
        attributes.append(attributes_)
        description.append(description_)
        url.append(url_)
        print(iterations)
    if iterations%100 == 0:
        print(str(iterations) + ' parsed succesfully')
        
    return title, price, neighborhood, attributes, description, url, model, vin, condition, cryptocurrency, delivery, engine_size, fuel_type, mileage, paint_color, street_legal, title_status, transmission, bike_type

SyntaxError: 'return' outside function (<ipython-input-24-f5ece5265499>, line 146)

In [23]:
motorcycle_post_parser()

0 parsed succesfully


([], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [])

In [13]:
title

NameError: name 'title' is not defined

In [ ]:
title = []

for _ in title15:
    for __ in (_):
#         __ = str(_[idx]).split('''>''')[0]
#         __ = str(_[idx]).split('''</''')[-2]
        title.append(__)

price = []

for _ in price15:
    for __ in _:
#         __ = str(_[idx]).split('''>''')[1]
#         __ = str(_[idx]).split('''</''')[-2]
        price.append(__)

neighborhood = []

for _ in neighborhood15:
    for __ in _:
#          __ = str(_[idx]).split('''(''')[1]
#         __ = str(_[idx]).split(''')''')[-2]
        neighborhood.append(__)

attributes = []

for _ in attributes15:
    for __ in _:
        attributes.append(__)

description = []

for _ in description15:
    for __ in _:
        description.append(__)

city = []

for _ in city15:
    for __ in _:
        city.append(__)